# Load and Preprocess Data

In [6]:
import librosa
import numpy as np
import os
DATASET_PATH = "../../dataset/Tess/"

confidence_mapping = {
    "happy": 1.0,
    "neutral": 0.8,
    "surprise": 0.7,
    "angry": 0.3,
    "fear": 0.2,
    "disgust": 0.1,
    "sad": 0.1
}

def extract_features(audio_path):
    y, sr = librosa.load(audio_path, sr=22050)

    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    pitch, _ = librosa.piptrack(y=y, sr=sr)
    loudness = np.mean(librosa.feature.rms(y=y))

    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr), axis=1)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y))

    features = np.hstack([
        np.mean(mfccs, axis=1), 
        np.std(mfccs, axis=1),
        np.mean(pitch),
        loudness,
        spectral_contrast,
        chroma,
        zero_crossing_rate
    ])
    return features


X, y, genders = [], [], []

for gender in ["Female", "Male"]:
    for emotion in confidence_mapping.keys():
        folder_path = os.path.join(DATASET_PATH, gender, emotion)
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            features = extract_features(file_path)
            X.append(features)
            y.append(confidence_mapping[emotion])
            genders.append(0 if gender == "Female" else 1)

X = np.array(X)
y = np.array(y)
genders = np.array(genders).reshape(-1, 1)


# Model

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Flatten, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

audio_input = Input(shape=(X.shape[1], 1), name="audio_input")
gender_input = Input(shape=(1,), name="gender_input") 


cnn = Conv1D(128, kernel_size=3, activation="relu", kernel_regularizer=l2(0.01))(audio_input)
cnn = LSTM(128, return_sequences=True)(cnn)
cnn = Dropout(0.3)(cnn)
cnn = LSTM(64, return_sequences=True)(cnn)
cnn = Dropout(0.3)(cnn)
cnn = LSTM(32)(cnn)
cnn = Dense(64, activation="relu")(cnn)
cnn = Dropout(0.3)(cnn)


merged = Concatenate()([cnn, gender_input])
output = Dense(1, activation="sigmoid", name="confidence_output")(merged)

model = Model(inputs=[audio_input, gender_input], outputs=output)
model.compile(optimizer="adam", loss="huber_loss", metrics=["mae"])


model.fit([X, genders], y, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
70/70 [==============================] - 4s 25ms/step - loss: 0.0681 - mae: 0.3186 - val_loss: 0.0907 - val_mae: 0.4158
Epoch 2/50
70/70 [==============================] - 1s 14ms/step - loss: 0.0585 - mae: 0.3181 - val_loss: 0.0968 - val_mae: 0.4370
Epoch 3/50
70/70 [==============================] - 1s 13ms/step - loss: 0.0563 - mae: 0.3151 - val_loss: 0.0849 - val_mae: 0.4094
Epoch 4/50
70/70 [==============================] - 1s 13ms/step - loss: 0.0555 - mae: 0.3135 - val_loss: 0.0846 - val_mae: 0.4088
Epoch 5/50
70/70 [==============================] - 1s 14ms/step - loss: 0.0547 - mae: 0.3120 - val_loss: 0.1008 - val_mae: 0.4467
Epoch 6/50
70/70 [==============================] - 1s 14ms/step - loss: 0.0542 - mae: 0.3103 - val_loss: 0.0962 - val_mae: 0.4363
Epoch 7/50
70/70 [==============================] - 1s 13ms/step - loss: 0.0537 - mae: 0.3086 - val_loss: 0.1003 - val_mae: 0.4456
Epoch 8/50
70/70 [==============================] - 1s 14ms/step - loss: 0.0533 - m